In [101]:
import xml.etree.ElementTree as Xet 
import pandas as pd 
import os
  

# scrape "date"
# scrape "Desc" -- differnent way DescriptionProgramSrvcAccomGrp is written

data = []
ns = "{http://www.irs.gov/efile}" # namespace
root_folder = '/Users/colden/Documents/VSCode/npo_classifier-master/My_Code/IRS_data_2018'

# parse XML file
for dirpath, dirnames, filenames in os.walk(root_folder):
    print(f"Dirpath: {dirpath}")
    print(f"Dirnames: {dirnames}")
    for file in filenames:
        if file == '.DS_Store': # probably a better way than skipping this file is to just use try/except or a check
            continue
        file_path = os.path.join(dirpath, file)
        xmlparsed = Xet.parse(file_path) 

        #xmlparsed = Xet.parse('/Users/colden/Documents/VSCode/npo_classifier-master/My_Code/2017_TEOS_XML_CT1/201701119349300915_public.xml')
        root = xmlparsed.getroot()

        EIN = root.find(ns+"ReturnHeader").find(ns+"Filer").find(ns+"EIN").text
        name = root.find(ns+"ReturnHeader").find(ns+"Filer").find(ns+"BusinessName").find(ns+"BusinessNameLine1Txt").text

        # if these are not set to none beforehand, errrors occur -- data understandably is appended at the end of the for loop
        desc1, desc2, desc3 = None, None, None
        descriptions = []
        try:
            program_accomplishments = root.find(ns+"ReturnData").find(ns+"IRS990EZ").findall(ns+"ProgramSrvcAccomplishmentGrp")
            descriptions = [program.find(ns+"DescriptionProgramSrvcAccomTxt").text for program in program_accomplishments]
            if len(descriptions) > 3:
                descriptions[2:] = [' '.join(descriptions[2:])] # if there are too many descriptions, combine them
            descriptions += [None] * (3 - len(descriptions))  # Fill the rest with None
            desc1, desc2, desc3 = descriptions
        except AttributeError:
            desc1, desc2, desc3 = None, None, None

        try:
            additional_desc = root.find(ns+"ReturnData").find(ns+"IRS990").find(ns+"Desc").text
        except AttributeError: additional_desc = "None"

        tax_period_begin = root.find(ns+"ReturnHeader").find(ns+"TaxPeriodBeginDt").text

        try:
            activity_or_mission = root.find(ns+"ReturnData").find(ns+"IRS990").find(ns+"ActivityOrMissionDesc").text
        except AttributeError: activity_or_mission = "None"

        try:
            mission_desc = root.find(ns+"ReturnData").find(ns+"IRS990").find(ns+"MissionDesc").text
        except AttributeError: mission_desc = "None"

        data.append({"EIN": EIN, "Name": name, "desc1": desc1, "desc2": desc2, "desc3" : desc3, "activity_or_mission": activity_or_mission, "mission_desc": mission_desc, "tax_period_begin": tax_period_begin, "additional_desc": additional_desc})

df = pd.DataFrame(data)
print(df.head())

df.to_csv('IRS_data_2018.csv')


Dirpath: /Users/colden/Documents/VSCode/npo_classifier-master/My_Code/IRS_data_2018
Dirnames: ['download990xml_2018_4', 'download990xml_2018_3', 'download990xml_2018_2', 'download990xml_2018_5', 'download990xml_2018_7', 'download990xml_2018_6', 'download990xml_2018_1']
Dirpath: /Users/colden/Documents/VSCode/npo_classifier-master/My_Code/IRS_data_2018/download990xml_2018_4
Dirnames: []
Dirpath: /Users/colden/Documents/VSCode/npo_classifier-master/My_Code/IRS_data_2018/download990xml_2018_3
Dirnames: []
Dirpath: /Users/colden/Documents/VSCode/npo_classifier-master/My_Code/IRS_data_2018/download990xml_2018_2
Dirnames: []
Dirpath: /Users/colden/Documents/VSCode/npo_classifier-master/My_Code/IRS_data_2018/download990xml_2018_5
Dirnames: []
Dirpath: /Users/colden/Documents/VSCode/npo_classifier-master/My_Code/IRS_data_2018/download990xml_2018_7
Dirnames: []
Dirpath: /Users/colden/Documents/VSCode/npo_classifier-master/My_Code/IRS_data_2018/download990xml_2018_6
Dirnames: []
Dirpath: /Users/